In [1]:
import gzip
import json
import pandas as pd
import numpy as np
import re
import ast

In [2]:
FILE_NAMES = ['.\\Datasets\\steam_games.json.gz','.\\Datasets\\user_reviews_fixed.json.gz','.\\Datasets\\users_items_fixed.json.gz']

In [6]:
steam_games_df = pd.read_json(FILE_NAMES[0],compression='gzip', lines=True)
steam_games_df

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [11]:
user_reviews_df = pd.read_json(FILE_NAMES[1],compression='gzip')
user_reviews_df

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [3]:
user_items_df = pd.read_json(FILE_NAMES[2], compression='gzip')
user_items_df

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864384,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712560,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445856,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099488,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066624,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700688,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759264,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548336,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [9]:
steam_games_df[steam_games_df.isnull().all(axis=1)]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88305,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
88306,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
88307,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
88308,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [12]:
# Dropeamos todas las filas de puros nulos
steam_games_df.dropna(how='all', inplace=True)
user_items_df.dropna(how='all', inplace= True)
user_reviews_df.dropna(how='all', inplace= True)

In [14]:
steam_games_df

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [ ]:
# Se detecta que app_name y title son básicamente lo mismo, se decide dropear title porque tiene vacios y app_name no
mask3 = steam_games_df[(steam_games_df['app_name'] != steam_games_df['title']) & (~steam_games_df['app_name'].isnull()) & (~steam_games_df['title'].isnull())]
mask3

In [ ]:
# Dropeamos columnas que no usaremos a futuro
# steam_Games | reviews_url , title
# user_reviews | user_url
# user_items | user_url
# 
steam_games_df = steam_games_df.drop('reviews_url', axis=1)
steam_games_df = steam_games_df.drop('title', axis=1)
user_items_df = user_items_df.drop('user_url',axis=1)
user_reviews_df = user_reviews_df.drop('user_url', axis=1)



In [ ]:
steam_games_df

In [ ]:
# Se Evalua que existen 2 juegos sin nombre se decide borrarlos
mask = steam_games_df['app_name'].isnull()
steam_games_df[mask]

steam_games_df = steam_games_df.dropna(subset=['app_name'])
steam_games_df[mask]

In [ ]:
steam_games_df

In [ ]:
steam_games_df = steam_games_df.explode(['genres'])
steam_games_df = steam_games_df.explode(['tags'])
steam_games_df = steam_games_df.explode(['specs'])

In [ ]:
dummies_genres = pd.get_dummies (steam_games_df['genres'], prefix='genres')
dummies_tags = pd.get_dummies (steam_games_df['tags'], prefix='tags')
dummies_specs = pd.get_dummies (steam_games_df['specs'], prefix='specs')


In [ ]:
steam_games_df = pd.concat([steam_games_df,dummies_genres,dummies_tags,dummies_specs]) # muy pesadp

In [ ]:
def convert_price(price):
    if price in ['Free Demo','Free to Play','Free To Play','Play for Free!','Free', 'Install Now', 'Play WARMACHINE: Tactics Demo', 
                 'Free Mod','Install Theme','Third-party','Play Now','Free HITMAN™ Holiday Pack','Install Now','Play the Demo','Free to Try',
                 'Free Movie','Free to Use']:
        return 0
    elif price == 'Starting at $499.00':
        return 499.00
    elif price == 'Starting at $449.00':
        return 449.00
    else:
        return price

steam_games_df.loc[:,'price'] = steam_games_df['price'].apply(convert_price)

In [ ]:
## Se decide en las fechas faltantes usar un mes y día standard 01/ 01 ya que solo necesitamos precisamente el valor del año

# def fix_release_date(date):

In [ ]:
#Checking Columns

publisher_data = steam_games_df['publisher'].unique()
# genres_data = steam_games_df['genres'].unique()
app_name_data = steam_games_df['app_name'].unique()
release_date_data = steam_games_df['release_date'].unique()

for i in release_date_data:
    print(i)

In [13]:
# steam_games_df.to_parquet('test.parquet')
steam_games_df.to_csv('games1.csv')

In [24]:
user_items_df

,user_id,items_count,steam_id,items
0,76561197970982479,277,76561197970982480,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864384,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712560,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445856,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099488,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...
88305,76561198323066619,22,76561198323066624,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700688,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759264,[]
88308,76561198329548331,7,76561198329548336,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [25]:
user_items_df.to_csv('user_items_df.csv')
user_items_df.to_parquet('user_items_df.parquet')


In [26]:
user_reviews_df.to_csv('user_reviews_df.csv')